# ORM using Peewee

ORM (Object Relational Mapper)  is a technique used in creating a "bridge" between object-oriented programs and, in most cases, relational databases.

Put another way, you can see the ORM as the layer that connects object oriented programming (OOP) to relational databases.

When interacting with a database using OOP languages, you'll have to perform different operations like creating, reading, updating, and deleting (CRUD) data from a database. By design, you use SQL for performing these operations in relational databases.

Class as defined in an Object Oriented (OO) programming language such as Python, is considered as non-scalar. It cannot be expressed as primitive types such as integers and strings.

On the other hand, databases like Oracle, MySQL, SQLite and others can only store and manipulate scalar values such as integers and strings organised within tables.

The programmer must either convert the object values into groups of scalar data types for storage in the database or convert them back upon retrieval, or only use simple scalar values within the program.

In an ORM system, each class maps to a table in the underlying database. Instead of writing tedious database interfacing code yourself, an ORM takes care of these issues, while you can focus on programming the logics of the system.

## Peewee

Peewee is a simple and small ORM. It has few (but expressive) concepts, making it easy to learn and intuitive to use.

* a small, expressive ORM
* python 2.7+ and 3.4+ (developed with 3.6)
* supports **sqlite**, **mysql**, **postgresql** and **cockroachdb**
* tons of extensions


**To Install:**

```python
pip install peewee
```

In [1]:
# Import Library
from peewee import (
    SqliteDatabase, 
    Model, 
    TextField,
    IntegerField)
import pandas as pd
import sqlite3

# Cleanup Database if exists
from pathlib import Path 
Path('database/sample_sqlite.db').unlink(missing_ok=True)

## Database
The Peewee `Database` object represents a connection to a database. The `Database` class is instantiated with all the information needed to open a connection to a database, and then can be used to:

* Open and close connections.
* Execute queries.
* Manage transactions (and savepoints).
* Introspect tables, columns, indexes, and constraints.

Peewee comes with support for SQLite, MySQL and Postgres. Each database class provides some basic, database-specific configuration options.

In [2]:
# Create Database via Peewee SqliteDatabase
sqlite_db = SqliteDatabase('database/sample_sqlite.db')
sqlite_db.connect()

True

## Create Table in SQLite

An object of Model sub class in Peewee API corresponds to a table in the database with which connection has been established. It allows performing database table operations with the help of methods defined in the Model class.

A user defined Model has one or more class attributes, each of them is an object of Field class. Peewee has a number of subclasses for holding data of different types. Examples are TextField, DatetimeField, etc. They correspond to the fields or columns in the database table. Reference of associated database and table and model configuration is mentioned in Meta class. Following attributes are used to specify configuration 

In [3]:
# Create Model that represent table
class Student(Model):
    name = TextField()
    age = IntegerField()
    
    class Meta:
        database = sqlite_db
        db_table = 'student'
        
# Create table using Model.create_table()
Student.create_table()

# This can also be done by using 
# sqlite_db.create_tables([Student])

Check whether user table is created

In [4]:
# Check table in database
!sqlite3 database/sample_sqlite.db .tables

student


Get Column name from student table to compare with class name

In [5]:
!sqlite3 database/sample_sqlite.db -cmd "PRAGMA table_info(student);" .exit

0|id|INTEGER|1||1
1|name|TEXT|1||0
2|age|INTEGER|1||0


Table student is created with same schema as Class Student


<br>

___

## Table Insertion

There are several method to insert new Rows to the table

In [6]:
# Insert new record using Object Creation with save() method
row1 = Student(name='John', age=20)
row1.save()

# Insert new record using Model Object with insert() method and execute()
row2 = Student.insert(name='Bob', age=22)
row2.execute()

# Insert new record using Model Object with create() method
Student.create(name='Jane', age=21)

<Student: 3>

### Bulk Inserts
In order to use multiple rows at once in the table, Peewee provides two methods: **bulk_create** and **insert_many**.

In [7]:
# Bulk Insert using Model.bulk_create()
a = Student(name='Alice', age=20)
b = Student(name='Ben', age=26)
Student.bulk_create([a, b])

In [8]:
# Bulk Insert using Model.insert_many()
rows = [
    {'name': 'Cindy', 'age': 25}, 
    {'name': 'David', 'age': 24}
]
Student.insert_many(rows).execute()

7

You can check now that those rows are already inserted

In [9]:
!sqlite3 database/sample_sqlite.db ".headers on" "SELECT * FROM student;"

id|name|age
1|John|20
2|Bob|22
3|Jane|21
4|Alice|20
5|Ben|26
6|Cindy|25
7|David|24


## Query Data using Peewee ORM

In [10]:
# Normal Query using default sqlite3
conn = sqlite3.connect('database/sample_sqlite.db')
query = conn.execute("SELECT * FROM student;").fetchall()
print('Total Rows: ', len(query))
query

Total Rows:  7


[(1, 'John', 20),
 (2, 'Bob', 22),
 (3, 'Jane', 21),
 (4, 'Alice', 20),
 (5, 'Ben', 26),
 (6, 'Cindy', 25),
 (7, 'David', 24)]

In [11]:
# Query using Peewee ORM to Dictionary
all_students = list(Student.select().dicts())
print('Total Rows: ', len(all_students))
all_students

Total Rows:  7


[{'id': 1, 'name': 'John', 'age': 20},
 {'id': 2, 'name': 'Bob', 'age': 22},
 {'id': 3, 'name': 'Jane', 'age': 21},
 {'id': 4, 'name': 'Alice', 'age': 20},
 {'id': 5, 'name': 'Ben', 'age': 26},
 {'id': 6, 'name': 'Cindy', 'age': 25},
 {'id': 7, 'name': 'David', 'age': 24}]

In [12]:
# Query using Peewee ORM to NamedTuple
all_students_tuple = list(Student.select().namedtuples())
print('Total Rows: ', len(all_students_tuple))
all_students_tuple

Total Rows:  7


[Row(id=1, name='John', age=20),
 Row(id=2, name='Bob', age=22),
 Row(id=3, name='Jane', age=21),
 Row(id=4, name='Alice', age=20),
 Row(id=5, name='Ben', age=26),
 Row(id=6, name='Cindy', age=25),
 Row(id=7, name='David', age=24)]

In [13]:
# Query using Peewee ORM to object
all_students_obj = list(Student.select())
print('Total Rows: ', len(all_students_obj))
all_students_obj

Total Rows:  7


[<Student: 1>,
 <Student: 2>,
 <Student: 3>,
 <Student: 4>,
 <Student: 5>,
 <Student: 6>,
 <Student: 7>]

In [14]:
# Inspection of object
first_row = all_students_obj[0]
print('Name: ', first_row.name)
print('Age: ', first_row.age)

Name:  John
Age:  20


## Select Filtering

In [15]:
# Query using SQL syntax
conn = sqlite3.connect('database/sample_sqlite.db')
query = conn.execute(
    "SELECT * FROM student WHERE age > 21;"
).fetchall()
query

[(2, 'Bob', 22), (5, 'Ben', 26), (6, 'Cindy', 25), (7, 'David', 24)]

In [16]:
# Query using Peewee ORM
orm_query = list(
    Student.select().where(Student.age > 21).dicts()
)
orm_query

[{'id': 2, 'name': 'Bob', 'age': 22},
 {'id': 5, 'name': 'Ben', 'age': 26},
 {'id': 6, 'name': 'Cindy', 'age': 25},
 {'id': 7, 'name': 'David', 'age': 24}]

In [17]:
# Convert to DataFrame
df = pd.DataFrame(orm_query)
df

,id,name,age
0,2,Bob,22
1,5,Ben,26
2,6,Cindy,25
3,7,David,24


---